In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install kneed

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt
from kneed import KneeLocator
from sklearn.metrics import silhouette_score
import seaborn as sns

In [ ]:
inpt1=pd.read_csv('/kaggle/input/ccdata/CC GENERAL.csv')
inpt1.head()

In [ ]:
inpt1.describe()

In [ ]:
inpt1.info()

In [ ]:
inpt1.columns

In [ ]:
####Droping the Null values as the number of count is too less
inpt1.drop('CREDIT_LIMIT', axis=1, inplace=True)
inpt1.drop('MINIMUM_PAYMENTS', axis=1, inplace=True)


In [ ]:
####Droping customer ID as its unique
inpt1.drop('CUST_ID', axis=1, inplace=True)

In [ ]:
####Check for skewness and colinearity#######
for cl in inpt1.columns:
    if inpt1[cl].dtype != 'object':
        sns.kdeplot(inpt1[cl])
        plt.show()




In [ ]:
for cl in inpt1.columns:
    inpt1[cl] = np.log(1 + inpt1[cl])

In [ ]:
####Check for skewness and colinearity#######
for cl in inpt1.columns:
    if inpt1[cl].dtype != 'object':
        sns.kdeplot(inpt1[cl])
        plt.show()


In [ ]:
###Visulizing the heatmap####
plt.figure(figsize=(12,12))
sns.heatmap(inpt1.corr(), annot=True)
plt.show()

In [ ]:
['BALANCE', 'BALANCE_FREQUENCY', 'PURCHASES', 'ONEOFF_PURCHASES',
       'INSTALLMENTS_PURCHASES', 'CASH_ADVANCE', 'PURCHASES_FREQUENCY',
       'ONEOFF_PURCHASES_FREQUENCY', 'PURCHASES_INSTALLMENTS_FREQUENCY',
       'CASH_ADVANCE_FREQUENCY', 'CASH_ADVANCE_TRX', 'PURCHASES_TRX',
       'PAYMENTS', 'PRC_FULL_PAYMENT', 'TENURE'],

In [ ]:
###Droping the highly postively coerelated columns####
inpt1.drop(['PURCHASES_TRX','INSTALLMENTS_PURCHASES','ONEOFF_PURCHASES','PURCHASES_FREQUENCY'], axis=1, inplace=True)

In [ ]:
####standarizing the data#####
ss=StandardScaler()
ss.fit(inpt1)
inpt1=ss.transform(inpt1)

**We are going to implement 3 clustering based algorthim and will evaulate all the 3 clustering based algorthims**

K-means

Using Elbow Method from visualization to find the optimum number of clusters

In [ ]:
wcss=[]
for i in range(2,20):
    knm=KMeans(n_clusters=i, random_state=0)
    knm.fit(inpt1)
    wcss_vl=knm.inertia_
    wcss.append(wcss_vl)
    print("Silhouette Coefficient for num_cluster",i, silhouette_score(inpt1, knm.labels_))
plt.xlabel('Number of Clusters')
plt.ylabel('WCSS')
plt.plot(np.arange(2,20), wcss)
plt.show()

**From the above graph it seems that an ideal cluster value would be some where arounf 8 or 9**

Using kneed library to automatically deduce the optimimum number of clusters

In [ ]:
wcss=[]
for i in range(1,20):
    knm=KMeans(n_clusters=i, random_state=0)
    knm.fit(inpt1)
    wcss_vl=knm.inertia_
    wcss.append(wcss_vl)
kneedle = KneeLocator(np.arange(1,20), wcss, S=1.0, curve='convex', direction='decreasing')
print('The elbow point for the curve is at cluster numeber',kneedle.knee)

# DBSCAN

In [ ]:
eps=np.arange(0.1, 0.8, 0.1)
min_samples=np.arange(5,13,1)
for ep, mns in zip(eps, min_samples):
    dbs=DBSCAN(eps=ep, min_samples=mns)
    dbs.fit(inpt1)
    print("Silhouette Coefficient for eps and min_samples",ep,mns,silhouette_score(inpt1, dbs.labels_))